<a href="https://colab.research.google.com/github/Natural-State/agol-data-workflows/blob/master/code/Colab%20notebooks/03_MCD64A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract date of fire burn per 500m pixel

## Import gee and authenticate

In [ ]:
import ee

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

## Input arguments for data extraction

In [ ]:
# Area of interest
aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/LLBN")
aoi_name = "LLBN"

# Layer
band_layer = "BurnDate"

# GEE layer ID
layer_name = "RS_004"

# Doesn't make sense to have reducer (just need to extract the max value for each year)

# Date parameters
start_year = 2012
end_year = 2022

# Range doesn't include the stop value
year_list = ee.List(list(range(start_year, end_year+1)))

## Import MCD64A1.061 MODIS Burned Area Monthly Global 500m



In [ ]:
modis = ee.ImageCollection("MODIS/061/MCD64A1")

## Burn date processing

In [ ]:
reducer_list = ee.Reducer.max()

def annualImage(year_date):
  start = ee.Date.fromYMD(year_date, 1, 1)
  end = ee.Date.fromYMD(year_date, 12, 31)
  date_range = ee.DateRange(start, end)
  name = start.format('YYYY_MM').cat('_to_').cat(end.format('YYYY_MM'))
  return modis \
        .filterDate(date_range) \
        .select(band_layer) \
        .reduce(reducer = reducer_list) \
        .clip(aoi) \
        .set({'name': name})

annual_image = year_list.map(annualImage)

## Check an element of list
year_mosaic  = ee.Image(annual_image.get(2))
label = ee.String(year_mosaic.get('name')).getInfo()
print(label)
print(year_mosaic.getInfo())

## Mapping images

SEE "GEE reducer tutorial" to define map at the begining so that can map layers in line with minimal code

In [ ]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

 # Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
  'min': 10,
  'max': 350,
  'palette': ['4e0400', '951003', 'c61503', 'ff1901']
  }

# Create a folium map object.
my_map = folium.Map(location=[0.1591, 37.3729], zoom_start=12)

# Add layers
for i in range(ee.List.length(annual_image).getInfo()):
  print(i)
  year_mosaic = ee.Image(annual_image.get(i))
  label = ee.String(year_mosaic.get('name')).getInfo()
  my_map.add_ee_layer(year_mosaic, vis_params, label)

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

## Export data - create task

`filenamePrefix` should be in format: place_layer_timeperiod

In [ ]:
for i in  range(ee.List.length(annual_image).getInfo()):

  output_img =  ee.Image(annual_image.get(i))
  output_name = f"{layer_name}_{aoi_name}_{ee.String(output_img.get('name')).getInfo()}"

  task = ee.batch.Export.image.toDrive(image = output_img,
                                     region = aoi.geometry().bounds(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = output_name,
                                     scale = 30,
                                     maxPixels = 10e12,
                                     crs = "EPSG:4326"
                                     )
  task.start()
  print("STARTED TASK ", i+1)

## Check task status

[List](https://developers.google.com/earth-engine/guides/processing_environments#list-of-task-states) of task status messages (state field)


In [ ]:
tasks = ee.batch.Task.list()
for task in tasks[0:ee.List.length(year_list).getInfo()]:
  task_id = task.status()['id']
  task_state = task.status()['state']
  print(task_id, task_state)

## Mount and unmount Google Drive


In [ ]:
# Mount
#from google.colab import drive

#ROOT = '/content/drive'
#drive.mount(ROOT, force_remount=True)


In [ ]:
# Unmount
#drive.flush_and_unmount()